<a href="https://colab.research.google.com/github/kaifkh20/nn_z_to_h/blob/main/nano_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt

--2025-05-06 08:52:39--  https://raw.githubusercontent.com/karpathy/char-rnn/refs/heads/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-05-06 08:52:39 (18.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset",len(text))

length of dataset 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
#store the data set encoded in a tensor
import torch
data = torch.tensor(encode(text),dtype = torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
#split 90-train 10-val
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
#sampling data of each block_size from random batch

@torch.no_grad()
def estimate_loss():
  out = {}
  m.eval()
  for split in ['train','val']:
    losses = torch.zeros(block_size)
    for k in range(block_size):
      X,Y = get_batch(split)
      logits,loss = m(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  m.train()
  return out

In [10]:
block_size = 8
train_data[:block_size+1].data
# decode(train_data[:block_size+1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


This trains in batches instead of training one by one of block size.

Instead of training like this:

[seq1] → loss → update weights  
[seq2] → loss → update weights  
[seq3] → ...

You train like this:

[seq1, seq2, seq3, seq4] → compute all losses together → average → update weights once


In [12]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  #generate a small batch of input x and target y
  ix = torch.randint(len(data)-block_size,(batch_size,)) #pick starting point for each from ex if 1000->data then pick a rand no till 992
  x = torch.stack([data[i:i+block_size]for i in ix]) #for each index i, take a slice of 8 tokens starting at position i from data.
  y = torch.stack([data[i+1:i+block_size+1]for i in ix]) # For the same indices, take the next 8 tokens, shifted by 1:
  return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("=====")

for b in range(batch_size): #batch dim
  for t in range(block_size): #block size
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[59, 52, 49, 47, 52, 42,  1, 40],
        [53, 54, 43, 44, 59, 50,  1, 50],
        [27, 24, 33, 25, 26, 21, 13, 10],
        [47, 41, 43,  1, 53, 60, 43, 56]])
targets:
torch.Size([4, 8])
tensor([[52, 49, 47, 52, 42,  1, 40, 56],
        [54, 43, 44, 59, 50,  1, 50, 39],
        [24, 33, 25, 26, 21, 13, 10,  0],
        [41, 43,  1, 53, 60, 43, 56, 58]])
=====
when input is [59] the target is 52
when input is [59, 52] the target is 49
when input is [59, 52, 49] the target is 47
when input is [59, 52, 49, 47] the target is 52
when input is [59, 52, 49, 47, 52] the target is 42
when input is [59, 52, 49, 47, 52, 42] the target is 1
when input is [59, 52, 49, 47, 52, 42, 1] the target is 40
when input is [59, 52, 49, 47, 52, 42, 1, 40] the target is 56
when input is [53] the target is 54
when input is [53, 54] the target is 43
when input is [53, 54, 43] the target is 44
when input is [53, 54, 43, 44] the target is 59
when input is [53, 54, 43, 44, 59] t

In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F

n_embd = 32
torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    self.lm_head = nn.Linear(n_embd,vocab_size)

  def forward(self,idx,targets=None):
    B,T = idx.shape
    tok_embd = self.token_embedding_table(idx) #B,T,C
    #We use a position embedding table to inject position information into the model
    #in which order does the word error
    #tok_embd only represent each word or token in n-d space without any order
    #pos_embd is helpful in enforcing order information
    pos_embd = self.position_embedding_table(torch.arange(T)) #T,C
    # print(pos_embd.shape,tok_embd.shape)
    x = tok_embd + pos_embd #B,T,C
    logits = self.lm_head(x) #B,T,vocab_size

    if targets is None:
      loss = None
    else:

      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)

      loss = F.cross_entropy(logits,targets)

    return logits,loss

  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self(idx)
      # Only keep the logits for the last token in the sequence
      # Shape goes from [batch_size, seq_len, vocab_size] → [batch_size, vocab_size]
    # This is because we only care about predicting the next token in the sequence
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim=-1)
      idx_next = torch.multinomial(probs,num_samples=1)
      idx = torch.cat((idx,idx_next),dim=1)

    return idx

m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb) # 4 batches each batch have block_size=8 each having prob=65


print(logits.shape)
print(loss)
# print(decode(m.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))
# print(out)


torch.Size([256, 65])
tensor(4.4529, grad_fn=<NllLossBackward0>)


In [14]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [15]:
batch_size = 32
for steps in range(10000):

  if steps % 1000 == 0:
    losses = estimate_loss()
    print(f"step {steps}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb,yb = get_batch('train')
  logits,loss = m(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

step 0: train loss 4.3963, val loss 4.3822
step 1000: train loss 2.5858, val loss 2.5586
step 2000: train loss 2.4885, val loss 2.5109
step 3000: train loss 2.4941, val loss 2.4312
step 4000: train loss 2.4657, val loss 2.4974
step 5000: train loss 2.4884, val loss 2.4611
step 6000: train loss 2.4562, val loss 2.4761
step 7000: train loss 2.4398, val loss 2.4400
step 8000: train loss 2.4606, val loss 2.4944
step 9000: train loss 2.4350, val loss 2.4823
2.42657732963562


In [16]:
print(decode(m.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))


Aye thim ne hise w, l pouthive! y:
Tour hastheaidilf, ary han, k man, w t t I m.
Ay y thind ilak, ther.
Gimar iowe; Bis ginere
NGoulumes the hertthensnsathor wed t
Sit winkite thitonshep h

Wo d INVERA I pa ite ho theres wo rors
MII oo to, st me intharis at ilpll o ine bl bandeno
MANolade aty gie; hakhinf me e mpesbeky It aimpromemurdou wht, t elly Tothas t kef owoo awis me a they coure henou ore
we at ing, hitongurrr,

Ane, areror rrtisar; hal ongerd, hatliortu hore tour me I tt he kn me tha x 


Mathematical trick in self-attention.

for every t, we need to calculate the mean upto t.
{because what we need is that every token t interacts with only its previous context tokens and not the future}

In [30]:
torch.manual_seed(1337)
B,T,C = 4,8,2 #batch,time,channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
#we want x[b,t] = mean(i<=t)
#version 1
B,T,C = 4,8,2
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1]
    xbow[b,t] = torch.mean(xprev,0)

xbow[1]

tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])

In [19]:
#version 2
wei = torch.tril(torch.ones(T,T)) #size is (T,T) for each token mean stored in each row till the token t
wei = wei/wei.sum(1,keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow,xbow2,atol=1e-6, rtol=1e-4)
# xbow2[1]

True

In [20]:
#version 3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
xbow3 = wei @ x
torch.allclose(xbow,xbow3,atol=1e-6, rtol=1e-4)

True

In [33]:
#version 4 self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32 #batch,time,channels
x = torch.randn(B,T,C)

#head self-attention
head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)
value = nn.Linear(C,head_size,bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2,-1)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)
out = wei @ x

out.shape
# torch.allclose(xbow,xbow3,atol=1e-6, rtol=1e-4)

torch.Size([4, 8, 32])

In [35]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2860, 0.7140, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3711, 0.3779, 0.2510, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2380, 0.2855, 0.1415, 0.3349, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0925, 0.0420, 0.0443, 0.7808, 0.0404, 0.0000, 0.0000, 0.0000],
        [0.0075, 0.0170, 0.0415, 0.0086, 0.3112, 0.6143, 0.0000, 0.0000],
        [0.0129, 0.0040, 0.0211, 0.8583, 0.0028, 0.0863, 0.0145, 0.0000],
        [0.0177, 0.0466, 0.0415, 0.6180, 0.0218, 0.1187, 0.1196, 0.0161]],
       grad_fn=<SelectBackward0>)